## Using AI for Anomalies Detection in Data Quality
**Description**: Implement an AI-based approach to detect anomalies in data quality.

**Steps**:
1. Use an Anomaly Detection Algorithm:
    - Use sklearn's Isolation Forest for anomaly detection.

**Example data:**

data = np.array([[25, 50000], [30, 60000], [35, 75000], [40, None], [45, 100000]])

2. Integrate with Great Expectations:
    - Generate alerts if anomalies are detected:

In [1]:
# Write your code from here
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.data_context import DataContext

data = np.array([[25, 50000], [30, 60000], [35, 75000], [40, None], [45, 100000]])
df = pd.DataFrame(data, columns=['age', 'income'])
df_cleaned = df.dropna()

model = IsolationForest(contamination='auto', random_state=42)
model.fit(df_cleaned)
predictions = model.predict(df_cleaned)
anomalies_indices = np.where(predictions == -1)[0]
anomalies = df_cleaned.iloc[anomalies_indices]

context = DataContext.create_ephemeral_data_context()

datasource_config = {
    "name": "pandas_datasource",
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "class_name": "PandasExecutionEngine",
        "module_name": "great_expectations.execution_engine",
    },
    "data_connectors": {
        "default_runtime_data_connector_name": {
            "class_name": "RuntimeDataConnector",
            "module_name": "great_expectations.datasource.data_connector",
            "batch_identifiers": ["default_identifier"],
        },
    },
}
context.add_datasource(**datasource_config)

batch_kwargs = {"default_identifier": "my_data", "data_asset_type": "DataFrame"}
context.add_pandas_df_to_datasource(df=df, datasource_name="pandas_datasource", batch_kwargs=batch_kwargs)
batch = context.get_batch(datasource_name="pandas_datasource", batch_kwargs=batch_kwargs)

if not anomalies.empty:
    print("Anomaly detected in data:")
    print(anomalies)
    expectation_configuration = ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_in_set",
        kwargs={
            "column": "age",
            "value_set": anomalies['age'].tolist()
        }
    )
    result = batch.validate(expectation_configuration=expectation_configuration)
    print(result)
    expectation_configuration = ExpectationConfiguration(
        expectation_type="expect_column_values_to_not_be_in_set",
        kwargs={
            "column": "income",
            "value_set": anomalies['income'].dropna().tolist()
        }
    )
    result = batch.validate(expectation_configuration=expectation_configuration)
    print(result)
else:
    print("No anomalies detected by Isolation Forest.")

ImportError: cannot import name 'contextfilter' from 'jinja2' (/home/vscode/.local/lib/python3.10/site-packages/jinja2/__init__.py)